# Parse YAML mapf files

In [2]:
using CRCBS.nusing LightGraphs, MetaGraphs, GraphUtils
using GraphPlottingBFS
using YAML

In [3]:
function generate_mapf_from_yaml(file_name)
    yaml_mapf = YAML.load(open(filename))
    # MAP
    dims = yaml_mapf["map"]["dimensions"]
    obstacles = [obs .+ [1,1] for obs in yaml_mapf["map"]["obstacles"]] # correct for 0-indexing 
    G, vtx_grid = initialize_grid_graph_with_obstacles(dims,obstacles)
    # AGENTS
    starts = [vtx_grid[(d["start"] .+ [1,1])...] for d in yaml_mapf["agents"]] # correct for 0-indexing 
    goals = [vtx_grid[(d["goal"] .+ [1,1])...] for d in yaml_mapf["agents"]] # correct for 0-indexing 
    mapf = initialize_mapf(
        GraphEnv.LowLevelEnv(graph=G),
        [GraphEnv.State(vtx=x,t=0) for x in starts],
        [GraphEnv.State(vtx=x) for x in goals]
    )
end

generate_mapf_from_yaml (generic function with 1 method)

In [ ]:
using DataStructures
function CRCBS.solve!(solver::CBSSolver, mapf::M where {M<:AbstractMAPF}, path_finder=a_star)
    # priority queue that stores nodes in order of their cost
    priority_queue = PriorityQueue{ConstraintTreeNode,Int}()

    root_node = initialize_root_node(mapf)
    low_level_search!(solver,mapf,root_node;path_finder=path_finder)
    detect_conflicts!(root_node.conflict_table,root_node.solution)
    @show is_valid(root_node.solution,mapf)
    @show get_final_state(root_node.solution[1])
    @show get_final_state(root_node.solution[2])
    if is_valid(root_node.solution,mapf)
        enqueue!(priority_queue, root_node => root_node.cost)
    end

    while length(priority_queue) > 0
        node = dequeue!(priority_queue)
        # check for conflicts
        conflict = get_next_conflict(node.conflict_table)
        if !is_valid(conflict)
            print("Optimal Solution Found! Cost = ",node.cost,"\n")
            return node.solution, node.cost
        end
        # otherwise, create constraints and branch
        constraints = generate_constraints_from_conflict(conflict)
        for constraint in constraints
            new_node = initialize_child_search_node(node)
            if add_constraint!(new_node,constraint)
                low_level_search!(solver, mapf, new_node,[get_agent_id(constraint)]; path_finder=path_finder)
                detect_conflicts!(new_node.conflict_table,new_node.solution,[get_agent_id(constraint)]) # update conflicts related to this agent
                if is_valid(new_node.solution, mapf)
                    enqueue!(priority_queue, new_node => new_node.cost)
                end
            end
        end
    end
    print("No Solution Found. Returning default solution")
    return default_solution(mapf)
end

In [ ]:
filename = "/home/kylebrown/Repositories/libMultiRobotPlanning/test/mapf_someAtGoal.yaml"
mapf = generate_mapf_from_yaml(filename)

solver = CBSSolver()
solution, cost = CRCBS.solve!(solver,mapf);

In [ ]:
paths = GraphEnv.convert_to_vertex_lists(solution)
render_grid_graph(G,[[s.vtx] for s in mapf.starts])

In [ ]:
for filename in readdir("/home/kylebrown/Repositories/libMultiRobotPlanning/test/")
    if splitext(filename)[end] == ".yaml"
        mapf = generate_mapf_from_yaml(filename)
        solver = CBSSolver()
        solution, cost = CRCBS.solve!(solver,mapf);
    end
end